These are systems with a contact Hamiltonian of the form
$$
H = ps + f(q)s + F(q, t)
$$

We will focus in particular on the Van der Pol oscillator
where $f(q) = - \epsilon (1-q^2)$ and $F(q) = q - \phi(t)$.

In [2]:
from integrators import contact as ic
from integrators.common import rk4
from lienard import integrator as li
from lienard import models as lm

def step6(system, dt, p, q, s, t, a=ic.a_six, stepper=li.step1):
    return ic.step6(system, dt, p, q, s, t, a=a, stepper=stepper)

def step6e(system, dt, p, q, s, t, a=ic.e_six, stepper=li.step1):
    return ic.step6(system, dt, p, q, s, t, a=a, stepper=stepper)

Lienard = lm.Lienard
VanDerPol = lm.VanDerPol
FritzhughNagumo = lm.FritzhughNagumo

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate as si
from mpl_toolkits import mplot3d
import progressbar

In [4]:
def err(x, y):
    errv = np.empty(len(x))
    for i in range(len(x)):
        errv[i] = max(errv[i-1] if i > 0 else 0, np.abs(x[i]-y[i]))
    return errv

def omega(t,e):
    o=((1 - e**2/16. + (17*e**4)/3072. + t**2/24. + (27*e**2*t**2)/128. +
    (781*e**4*t**2)/73728. + (3*t**4)/640. + (149*e**2*t**4)/2048. -
    (339041*e**4*t**4)/3.538944e6 + (5*t**6)/7168. + (559*e**2*t**6)/16384. +
    (4695149*e**4*t**6)/8.4934656e7))/(2*np.pi)
    return o

## Attraction Basin

In [ ]:
epsilon=50.
xmin, xmax = -10.,10.
ymin, ymax = -100.,100.
gridsize = 0.1
delta=1e-5
gridx=np.arange(xmin,xmax,gridsize)
gridy=np.arange(ymin,ymax,gridsize)
nx,ny=len(gridx),len(gridy)
color=np.zeros((nx,ny))
test=True
vdp=VanDerPol(epsilon, 0, 0)
p0=0
tspan1=np.arange(0.,8.,0.01) #short integration for the limit cycle
tspan=np.arange(0.,100.,0.01) #long integration for the graphics

#limitcycle
p0, q0, s0 = 0, 2, 0
cicl,cicls,_= ic.integrate(li.step1, vdp, tspan1, p0, q0, s0)
equi=np.array([[0,0]])
with progressbar.ProgressBar(max_value=nx*ny) as bar:
    s=0
    for i in range(nx):
        for j in range(ny):
            s+=1
            bar.update(s)
            q0,s0=gridx[i],gridy[j]
            sol, sols, _ = ic.integrate(li.step1, vdp, tspan, p0, q0, s0)
            finpt=np.array([sol[len(tspan)-1,1],sols[len(tspan)-1]])
            for t in range(len(tspan1)):
                if test and np.linalg.norm(finpt-np.array([cicl[t,1],cicls[t]]))<delta:
                    color[i,j]=-1
                    test=False
            for t in range(len(equi)):
                if test:
                    if np.linalg.norm(finpt-equi[t])<delta:
                        color[i,j]=t
                        test=False
            if test:
                equi=np.vstack([equi,finpt])
                color[i,j]=len(equi)-1

plt.figure(figsize=(30,30))
plt.imshow(color, cmap=plt.cm.BuPu_r)
plt.show()